# Crew AI

In [1]:
import os
import crewai
from crewai import Agent, Task, Crew, Process
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import PyPDF2

load_dotenv(override=True)

OPENAI_MODEL_NAME = os.getenv('OPENAI_API_TYPE')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
AZURE_OPENAI_LLM_DEPLOYMENT_NAME = os.getenv('AZURE_OPENAI_LLM_DEPLOYMENT_NAME')
AZURE_OPENAI_LLM_MODEL_NAME = os.getenv('AZURE_OPENAI_LLM_MODEL_NAME')           


In [2]:
llm = AzureChatOpenAI(
    azure_deployment=AZURE_OPENAI_LLM_DEPLOYMENT_NAME, 
    model=AZURE_OPENAI_LLM_MODEL_NAME,
    temperature=0.8
)


In [3]:
# PDF extraction function
def extract_text_from_pdf(pdf_file_path):
    text = ""
    with open(pdf_file_path, 'rb') as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page in range(len(reader.pages)):
            text += reader.pages[page].extract_text()
    return text

![Agents](img/AI_Crew_Agent.png)

In [4]:
# Agents 
good_example_analyzer = Agent(
    role="Good Example Analyzer",
    goal="Analyze the good example ticket to extract the style, structure, and key elements expected for a well-formed user story.",
    llm=llm,
    backstory="You are an experienced product owner and Agile coach. Your job is to understand how a well-written user story is structured and provide insights on style, structure, and tone to the user story generator."
)

feature_analyzer = Agent(
    role="Feature Analyzer",
    goal="Analyze the feature description and extract key information including context, problem statement, target audience, and goals.",
    llm=llm,
    backstory="You are an experienced business analyst with deep expertise in software requirements. Your role is to extract critical information from technical documentation and feature descriptions, identifying the key objectives, problems being solved, and the relevant stakeholders."
)

ticket_parser = Agent(
    role="Ticket Parser",
    goal="Extract the ticket name, short description, and additional metadata such as dependencies, links, epics, and sprints from the user story ticket.",
    llm=llm,
    backstory="You are detail-oriented and skilled at parsing technical documents. Your goal is to capture all critical information from the ticket, including dependencies, links, and other metadata required to complete the story."
)

user_story_generator = Agent(
    role="User Story Generator",
    goal="Generate a well-structured user story, including detailed descriptions and acceptance criteria, based on the provided feature summary and ticket information. Ensure missing information is identified and clearly prompt for clarification.",
    llm=llm,
    backstory="You are an expert in Agile development with a deep understanding of best practices for writing user stories. Your role is to ensure user stories are complete, concise, and include well-defined acceptance criteria, highlighting any gaps or ambiguities that need further clarification from stakeholders."
)


![Tasks](img/AI_Crew_Tasks.png)

In [5]:
# Tasks

analyze_good_example_task = Task(
    description="""
    Analyze the following well-written ticket, and extract:
    * The expected style (e.g., formal, clear, concise)
    * The structure (e.g., clear definition of role, goal, and benefit)
    * Important elements in the user story (e.g., specific format, clarity, completeness)
    
    Provide insights to the user story generator so that it can replicate the style and structure in future user stories.
    """,
    expected_output="Insights on style, structure, and key elements from the good example ticket.",
    agent=good_example_analyzer,
    async_execution=True
)

analyze_feature_task = Task(
    description="""
    Analyze the feature description and extract the following:
    * Context: What broader system or environment does this feature exist in?
    * Problem Statement: What specific issue or need is the feature addressing?
    * Target group and stakeholders: Who are the users or stakeholders involved?
    * Goals/Benefit: What are the intended outcomes or benefits of this feature?
    """,
    expected_output="A structured summary of the feature’s key points, including context, problem statement, target audience, and goals.",
    agent=feature_analyzer,
    async_execution=True
)

parse_ticket_task = Task(
    description=f"""
    Analyze the following ticket text and extract the following:
    * Ticket Name
    * Short Description
    * Dependencies (related tickets, features, epics, etc.)
    * Links (Confluence, Jira, or other relevant links)
    * Metadata (sprint information, epic links, affected versions)
    """,
    expected_output="A structured format containing the ticket name, short description, dependencies, links, and additional metadata.",
    agent=ticket_parser,
    async_execution=True
)

generate_user_story_task = Task(
    description=f"""
    Based on the insights from the good example ticket, feature text, and ticket information, generate:

    * A well-formed user story in the format: "As a <user role>, I want <goal/desire> so that <reason/benefit>."
    * A detailed description that incorporates the feature’s context, addressing any missing or unclear information.
    * Clear and specific acceptance criteria that cover functional, non-functional, and error-handling aspects.
    * Identify information that is missing or unclear, and explicitly prompt for clarification from stakeholders (e.g., "Ask the stakeholder which data should be archived").
    * Seamlessly integrate all relevant metadata (dependencies, links, epics, etc.) into the user story, description, and acceptance criteria without listing them as a separate section.
    """,
    expected_output="A well-structured user story with integrated metadata and prompts for clarification where needed.",
    agent=user_story_generator
)


In [6]:
def process_pdf_tasks(feature_pdf, ticket_pdf, good_example_pdf):
    
    feature_text = extract_text_from_pdf(feature_pdf)
    ticket_text = extract_text_from_pdf(ticket_pdf)
    good_example_text = extract_text_from_pdf(good_example_pdf)
    
    if not feature_text or not ticket_text or not good_example_text:
        raise ValueError("PDF extraction failed: One or more PDFs returned empty text.")

    # Update task descriptions with extracted text
    analyze_feature_task.description = f"Analyze the following feature text:\n{feature_text}"
    parse_ticket_task.description = f"Analyze the following ticket text and extract dependencies, links, and metadata:\n{ticket_text}"
    analyze_good_example_task.description = f"Analyze the following good example ticket:\n{good_example_text}"

    crew = Crew(
        agents=[feature_analyzer, ticket_parser, user_story_generator, good_example_analyzer],
        tasks=[analyze_feature_task, parse_ticket_task, analyze_good_example_task, generate_user_story_task]
    )

    result = crew.kickoff()
    
    
    return result


In [ ]:
feature_pdf_path = ['FEATURE PDF PATH']
ticket_pdf_path = ['TICKET PDF PATH']
good_example_pdf_path = ['EXAMPLE PDF PATH']

result = process_pdf_tasks(feature_pdf_path, ticket_pdf_path, good_example_pdf_path)

print(result)